In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
from scipy import linalg

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
def importFlux():
    fhcFile = "/Users/elenag/Desktop/PlotterLEE/NuMIFlux/NuMIFluxBeamVar/BeamlineGeometryVariations/FHC/output_uboone_fhc_run0_merged.root"
    rhcFile = "/Users/elenag/Desktop/PlotterLEE/NuMIFlux/NuMIFluxBeamVar/BeamlineGeometryVariations/RHC/output_uboone_rhc_run0_merged.root"
    fhcHistoNue     = uproot.open(fhcFile)['nue/Detsmear/nue_CV_AV_TPC_5MeV_bin']
    rhcHistoNue     = uproot.open(rhcFile)['nue/Detsmear/nue_CV_AV_TPC_5MeV_bin']
    fhcHistoNueBar  = uproot.open(fhcFile)['nuebar/Detsmear/nuebar_CV_AV_TPC_5MeV_bin']
    rhcHistoNueBar  = uproot.open(rhcFile)['nuebar/Detsmear/nuebar_CV_AV_TPC_5MeV_bin']
    print(rhcHistoNue.bins[len(rhcHistoNue.bins)-1], rhcHistoNueBar.bins[len(rhcHistoNue.bins)-1])
    #print(fhcHistoNue.bins, fhcHistoNueBar.bins)

importFlux()

In [ ]:
#####################################################################################
############################### MC as if it was data ################################
#####################################################################################
tree = "NeutrinoSelectionFilter"

NU_R1    = 'test/prodgenie_numi_uboone_overlay_fhc_mcc9_run1_v28_all_snapshot'    +ls.APPEND #newTune
NUE_R1   = '/Users/elenag/Desktop/PlotterLEE/root_files/FullMC/neutrinoselection_filt_run1_overlay_intrinsic'+ls.APPEND #newTune

mc_R1    = uproot.open(ls.ntuple_path+NU_R1 +".root")[ls.fold][tree]
nue_R1   = uproot.open(NUE_R1+".root")[ls.fold][tree]

variables = ["run", "sub", "evt", "nu_pdg","shr_tkfit_nhits_Y", "shr_tkfit_nhits_U", "shr_tkfit_nhits_V", "shr_tkfit_dedx_Y", 
             "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", 
             "shr_tkfit_2cm_nhits_Y", "shr_tkfit_2cm_nhits_U", "shr_tkfit_2cm_nhits_V", 
             "shr_tkfit_2cm_dedx_Y", "shr_tkfit_2cm_dedx_U", "shr_tkfit_2cm_dedx_V",             
             "shr_energy_tot_cali", "trk_energy_tot", "shr_energy", "shr_theta", "trk_theta_v", "trk_phi_v", 
             "trk_score_v", "trk_llr_pid_score_v", "topological_score","trk_energy_proton_v", "trk_calo_energy_y_v", "trk_id", 
             "n_tracks_contained", "shrsubclusters0", "shrsubclusters1", "shrsubclusters2", 
             "shr_tkfit_npointsvalid", "shr_tkfit_npoints", "secondshower_Y_dir", "shrclusdir2", "trk_theta", 
             "trk_sce_start_x_v", "trk_sce_start_y_v", "trk_sce_start_z_v", 
             "trk_sce_end_x_v", "trk_sce_end_y_v", "trk_sce_end_z_v", 
             "shrmoliereavg", "secondshower_Y_dot", "true_e_visible",
             "pfnhits", "pfnunhits", "flash_time", "category", "shr_score", "tksh_distance","interaction", "tksh_angle",
             "trkshrhitdist2", "hits_ratio", "secondshower_Y_nhit", "secondshower_Y_vtxdist", "CosmicIPAll3D", 
             "CosmicDirAll3D", "nslice", "selected", "_opfilter_pe_beam", "_opfilter_pe_veto", "n_showers_contained", 
             "reco_nu_vtx_sce_x", "reco_nu_vtx_sce_y", "reco_nu_vtx_sce_z", 
             "pt", "shr_phi", "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
             "shr_tkfit_gap10_dedx_Y", "shr_tkfit_gap10_dedx_U", "shr_tkfit_gap10_dedx_V",
             "slnunhits","slnhits","pi0_e","ccnc","shr_pz","shr_py","shr_px","npi0"]

varMC = ["nu_e","true_pt","true_pt_visible","true_p","true_p_visible","true_nu_vtx_t","true_nu_vtx_x","true_nu_vtx_y","true_nu_vtx_z","true_nu_vtx_sce_x","true_nu_vtx_sce_y","true_nu_vtx_sce_z\
","true_nu_px","true_nu_py","true_nu_pz"]
'''
variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "flash_time",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    "topological_score",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", 
    "run", "sub", "evt", 
    "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    "pfnhits","pfnunhits",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v","shr_id", 
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U",
    "pi0_dir2_x","pi0_dir2_y","pi0_dir2_z","pi0_dir1_x","pi0_dir1_y","pi0_dir1_z",
    "pi0truth_gamma1_etot","pi0truth_gamma2_etot","pi0truth_gammadot","pi0truth_gamma_parent",
    "trk_theta_v","trk_phi_v"
]
'''

#make the list unique
variables = list(set(variables))
print(variables)


    
WEIGHTS     = ["weightSpline","weightTune","weightSplineTimesTune","ppfx_cv"]
SYSTEMATICS = ['weightsFlux','weightsGenie','weightsReint'] 
PPFX        = ['weightsPPFX'] 



nue_R1  = nue_R1.pandas.df(variables + WEIGHTS + SYSTEMATICS + PPFX, flatten=False)
mc_R1   = mc_R1.pandas .df(variables + WEIGHTS + SYSTEMATICS + PPFX, flatten=False)


In [ ]:
df_v =  [mc_R1,nue_R1]
#removeGenieTune = False
for i,df in enumerate(df_v):
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.    
    
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1. 

    df.loc[ df['weightSpline'] <= 0, 'weightSpline' ] = 1.
    df.loc[ df['weightSpline'] == np.inf, 'weightSpline' ] = 1.
    df.loc[ df['weightSpline'] > 100, 'weightSpline' ] = 1.
    df.loc[ np.isnan(df['weightSpline']) == True, 'weightSpline' ] = 1. 
    
    df[ 'weightSplineTimesTuneTimesPPFX' ] = df[ 'weightSplineTimesTune' ]*df['ppfx_cv']
    df[ 'weightSplineTimesPPFX' ]          = df[ 'weightSpline' ]*df['ppfx_cv']
    
    # Cray cray weight fix
    for ievt in range(df.shape[0]):
        reweightCondition = ((df['weightsGenie'].iloc[ievt] > 3000) | (df['weightsGenie'].iloc[ievt] < 0)  | 
                             (df['weightsGenie'].iloc[ievt] == np.inf))
        df['weightsGenie'].iloc[ievt][ reweightCondition ] = 1.



In [ ]:
# add back the cosmic category, for background only
df_v = [mc,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

# Plotting part

In [ ]:
import plotter
import importlib
importlib.reload(plotter)


In [ ]:
import plotter
import importlib
importlib.reload(plotter)
scaling = 1

SPLIT = 1.0
#nueMC = mc_R1.query("(abs(nu_pdg) == 12 & ccnc == 0)")

nue_R1   = nue_R1.query("(abs(nu_pdg) == 12 & ccnc == 0 & true_nu_vtx_z < 1036.8 & true_nu_vtx_z > 0 & true_nu_vtx_y < 116.5 & true_nu_vtx_y > -116.5 & true_nu_vtx_x < \
 254.8 & true_nu_vtx_x > -1.55)")
nue_R3   = nue_R1.query("(abs(nu_pdg) == 12 & ccnc == 0 & true_nu_vtx_z < 1036.8 & true_nu_vtx_z > 0 & true_nu_vtx_y < 116.5 & true_nu_vtx_y > -116.5 & true_nu_vtx_x < \
 254.8 & true_nu_vtx_x > -1.55)")


data =  nue_R1.query("evt == 1802") 
dirt =  nue_R1.query("evt == 1802") 
ext  =  nue_R1.query("evt == 1802") 

samples_R1 = {
    "mc"  : mc_R1,
    "nue" : nue_R1,
    "data": data,
    "dirt": dirt,
    "ext" : ext
}


samples_R3 = {
    "mc"  : mc_R3,
    "nue" : nue_R3,
    "data": data,
    "dirt": dirt,
    "ext" : ext
}


pot_R1       = 2e+20
mcratio_R1   = pot_R1/2.32135e+21 
nueratio_R1  = pot_R1/2.59311e+22 
dirtratio_R1 = pot_R1/1.42143e+21
extratio_R1  = 5268051/9199232.74 #partial file: 2347745.0/4015961.99


pot_R3        = 5.0e20 
mcratio_R3    = pot_R3/1.99076e+21
nueratio_R3   = pot_R3/2.5392e+22 
dirtratio_R3  = pot_R3/1.03226e+21
extratio_R3   = 10363728.0/32878305.25 

print(nueratio_R1, nueratio_R3)

scaling    = 1
SPLIT      = 1.0
scalingEXT = 0.98
scalingDRT = 0.35

weights_R1 = {
            "mc"  : mcratio_R1   * SPLIT * scaling,   
            "nue" : nueratio_R1  * SPLIT * scaling,  
            "ext" : extratio_R1  * SPLIT * scalingEXT, 
            "dirt": dirtratio_R1 * SPLIT * scalingDRT, 
}
my_plotter_R1 = plotter.Plotter(samples_R1, weights_R1, pot=pot_R1)


weights_R3 = {
            "mc"  : mcratio_R3   * SPLIT * scaling,   
            "nue" : nueratio_R3  * SPLIT * scaling,  
            "ext" : extratio_R3  * SPLIT * scalingEXT, 
            "dirt": dirtratio_R3 * SPLIT * scalingDRT, 
}
my_plotter_R3 = plotter.Plotter(samples_R3, weights_R3, pot=pot_R3)


In [ ]:
QUERY = 'abs(nu_pdg) == 12'

VARIABLE, BINS, RANGE, XTIT = 'nproton', 8, (0.0,8.), r"n protons"
VARIABLE, BINS, RANGE, XTIT = 'nu_e', 100, (0.0,20.), r"Reconstructed Energy [GeV]"

In [ ]:
sysBool = False
import matplotlib.pyplot as plt
import matplotlib

fig, ax1, ax2, label, puppa = my_plotter_R3.plot_variable(
    VARIABLE,   
    query= QUERY ,#BDTCQ,#NPLCUTQ, #QUERY, #'nslice == 1', #QUERY, #+" and category != 111",
    kind="event_category",
    draw_sys=sysBool,
    draw_data=False,
    draw_geoSys = sysBool, 
    stacksort=3,
    genieweight = "weightSplineTimesTuneTimesPPFX", #"weightSplineTimesTuneTimesPPFX", 
    title=XTIT,
    bins=BINS,
    range=RANGE #,
)[0:5]


#ax1.scatter(midBin,label[0][1])
midBin = (label[1][1:] + label[1][:-1])/2.
rate_R3 = label[0][1]

In [ ]:
fig, ax1, ax2, label, puppa = my_plotter_R1.plot_variable(
    VARIABLE,   
    query= QUERY ,#BDTCQ,#NPLCUTQ, #QUERY, #'nslice == 1', #QUERY, #+" and category != 111",
    kind="event_category",
    draw_sys=sysBool,
    draw_data=False,
    draw_geoSys = sysBool, 
    stacksort=3,
    genieweight = "weightSplineTimesTuneTimesPPFX", #"weightSplineTimesTuneTimesPPFX", 
    title=XTIT,
    bins=BINS,
    range=RANGE #,
)[0:5]

rate_R1 = label[0][1]

In [ ]:
# Super Spherical Cow
# N Events = Flux [/cm2/sec]* XS [cm2] * Targets
def solveForXS(Flux_nu_FHC,Flux_nu_RHC,Flux_antinu_FHC,Flux_antinu_RHC,Rate_FHC,Rate_RHC):
    a = np.array([[Flux_nu_FHC, Flux_nu_RHC], [Flux_antinu_FHC, Flux_antinu_RHC]])
    b = np.array([Rate_FHC, Rate_RHC])
    from scipy import linalg
    x = linalg.solve(a, b)
    return(x)



In [ ]:
def importFlux():
    fhcFile = "/Users/elenag/Desktop/PlotterLEE/NuMIFlux/NuMIFluxBeamVar/BeamlineGeometryVariations/FHC/output_uboone_fhc_run0_set1.root"
    rhcFile = "/Users/elenag/Desktop/PlotterLEE/NuMIFlux/NuMIFluxBeamVar/BeamlineGeometryVariations/RHC/output_uboone_rhc_run0_set1.root"
    fhcHistoNue     = uproot.open(fhcFile)['nue/Detsmear/nue_CV_AV_TPC_5MeV_bin']
    rhcHistoNue     = uproot.open(rhcFile)['nue/Detsmear/nue_CV_AV_TPC_5MeV_bin']
    fhcHistoNueBar  = uproot.open(fhcFile)['nuebar/Detsmear/nuebar_CV_AV_TPC_5MeV_bin']
    rhcHistoNueBar  = uproot.open(rhcFile)['nuebar/Detsmear/nuebar_CV_AV_TPC_5MeV_bin']
    potRHC_MC = 465.
    potFHC_MC = 491.
    #print(rhcHistoNue.values[0], rhcHistoNueBar.bins[0])
    #ReBin the bins
    v = np.arange(0,4000)
    v = np.hstack([v[::40],v[39::40]])
    v.sort()
    
    # Normalized to MC POT
    valuesRHC_Nue    = (rhcHistoNue.values.reshape((100,40))).sum(axis=1)/potRHC_MC
    valuesRHC_NueBar = (rhcHistoNueBar.values.reshape((100,40))).sum(axis=1)/potRHC_MC
    valuesFHC_Nue    = (fhcHistoNue.values.reshape((100,40))).sum(axis=1)/potFHC_MC
    valuesFHC_NueBar = (fhcHistoNueBar.values.reshape((100,40))).sum(axis=1)/potFHC_MC

    #Same Bins for all histograms
    fluxBins = rhcHistoNue.bins[v,[0,1]*100].reshape((100,2))
    return(fluxBins, valuesFHC_Nue, valuesFHC_NueBar, valuesRHC_Nue, valuesRHC_NueBar)

fluxTuple = importFlux()

In [ ]:
fluxB            = fluxTuple[0]
valuesFHC_Nue    = fluxTuple[1]
valuesFHC_NueBar = fluxTuple[2]
valuesRHC_Nue    = fluxTuple[3]
valuesRHC_NueBar = fluxTuple[4]

print(rate_R1.shape,rate_R3.shape,midBin.shape)
print(fluxB.shape,valuesFHC_Nue.shape,valuesFHC_NueBar.shape)
print(fluxB.shape,valuesRHC_Nue.shape,valuesRHC_NueBar.shape)


for i in range(99):
    if valuesFHC_Nue[i] == 0:
        break
    print(i, solveForXS(valuesFHC_Nue[i],valuesRHC_Nue[i],valuesFHC_NueBar[i],valuesRHC_NueBar[i],rate_R1[i],rate_R3[i]))